In [15]:
import pandas as pd
import folium
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
import datetime

In [2]:
position_records = []

directory = os.fsencode('vehicle_positions_hist')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    with open(f'vehicle_positions_hist/{filename}', 'r') as f:
        new_records = json.load(f)
        for record in new_records:
            record['insert_ts'] = int(filename.replace('.json', ''))

    position_records.extend(new_records)

In [3]:
df = pd.DataFrame.from_records(position_records)

routes = pd.read_csv('routes.txt')

df = df.merge(routes[['route_id', 'route_color']], on='route_id', how='left')
df['route_color'] = df['route_color'].fillna('0076CE')
df['route_color'] = '#' + df['route_color']

df['vehicle_update_ts_rounded'] = (np.ceil(df['vehicle_update_ts'].fillna(0) / 30)).astype('int') * 30

In [4]:
rail_route_ids = ['101D', '101E', '101H', '103W', '107R', '109L', '113B', '113G', '117N', 'A', 'FF1', 'FF2', 'FF3', 'AB1']

df['size'] = 3
df.loc[df['route_id'].isin(rail_route_ids), 'size'] = 10

all_timestamps = df['vehicle_update_ts_rounded'].unique()

In [5]:
def get_filled_vehicle_df(vehicle_id):
    veh_df = df[df['vehicle_id'] == vehicle_id].sort_values('vehicle_update_ts_rounded').drop_duplicates()
    veh_df['row_num'] = veh_df.sort_values(['vehicle_update_ts_rounded','insert_ts'], ascending=[True,True]).groupby(['vehicle_update_ts_rounded']).cumcount() + 1
    veh_df = veh_df[veh_df['row_num'] == 1]

    veh_df = veh_df.set_index(veh_df['vehicle_update_ts_rounded'])
    veh_df.sort_index(inplace=True)

    new_index = pd.Index(np.arange(all_timestamps.min(), all_timestamps.max(), 30), name='vehicle_update_ts_rounded_full')
    veh_df = veh_df.reindex(new_index, method='pad').reset_index()

    veh_df = veh_df[veh_df['vehicle_update_ts_rounded_full'].between(veh_df['vehicle_update_ts_rounded'].min(), veh_df['vehicle_update_ts_rounded'].max())]

    return veh_df

In [6]:
filled_df = pd.concat([get_filled_vehicle_df(vehicle_id) for vehicle_id in df['vehicle_id'].unique()])

In [41]:
filled_df.loc[~filled_df['route_id'].isin(rail_route_ids), 'size'] = 3

ts_ex = filled_df['vehicle_update_ts_rounded_full'].unique()[1000]
plot_df = filled_df[(filled_df['vehicle_update_ts_rounded_full'] == ts_ex)]

plot_df

,vehicle_update_ts_rounded_full,vehicle_update_ts,lat,lon,trip,route_id,stop_id,vehicle_id,vehicle_label,insert_ts,route_color,vehicle_update_ts_rounded,size,row_num
2169,1718296710,1.718296e+09,39.765999,-104.850632,114904158,153,34738,1A5C9D8E170E9DD4E063DD4D1FAC509B,6094,1.718296e+09,#0076CE,1.718296e+09,3.0,1.0
2169,1718296710,1.718297e+09,39.739834,-104.865898,114899868,105,14257,1A5C9D8E17149DD4E063DD4D1FAC509B,6100,1.718297e+09,#0076CE,1.718297e+09,3.0,1.0
2169,1718296710,1.718297e+09,39.666218,-104.831383,114913713,AT,26198,1A5C9D8E15B39DD4E063DD4D1FAC509B,1506,1.718297e+09,#57C1E9,1.718297e+09,3.0,1.0
2169,1718296710,1.718297e+09,40.154732,-105.102631,114913847,BOLT,17677,1A5C9D8E15D69DD4E063DD4D1FAC509B,1538,1.718297e+09,#64A70B,1.718297e+09,3.0,1.0
2169,1718296710,1.718295e+09,39.912582,-104.993614,114899529,104L,26109,1A5C9D8E15E39DD4E063DD4D1FAC509B,1551,1.718295e+09,#0076CE,1.718295e+09,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,1718296710,1.718293e+09,39.655117,-104.990585,,,,1A5C9D8E16ED9DD4E063DD4D1FAC509B,6059,1.718293e+09,#0076CE,1.718293e+09,3.0,1.0
2169,1718296710,1.718295e+09,40.017117,-105.276436,,,,1A5C9D8E167F9DD4E063DD4D1FAC509B,3708,1.718295e+09,#0076CE,1.718295e+09,3.0,1.0
2169,1718296710,1.718294e+09,40.149166,-105.117180,,,,1A5C9D8E196F9DD4E063DD4D1FAC509B,6616,1.718294e+09,#0076CE,1.718294e+09,3.0,1.0
2169,1718296710,1.718294e+09,39.752132,-104.997185,,,,1A5C9D8E1A1E9DD4E063DD4D1FAC509B,9394,1.718295e+09,#0076CE,1.718294e+09,3.0,1.0


In [50]:
len( filled_df['vehicle_update_ts_rounded_full'].unique())

3326

In [42]:
m = folium.Map(location=(39.8, -105), tiles="cartodb positron", zoom_start=11, width=1440, height=1440)



for index, row in plot_df.iterrows():
    folium.CircleMarker(location=[row.lat, row.lon], fill=True, stroke=False, radius=row['size'], color=row.route_color, fill_opacity=0.6).add_to(m)

m

In [49]:
m.save('folium.html')

In [56]:
def export_image_smooth(timestamp):
    plot_df = filled_df[(filled_df['vehicle_update_ts_rounded_full'] == timestamp)]
    fig = px.scatter_mapbox(plot_df, lat='lat', lon='lon', height=1440, width=1440, center={'lat': 39.9, 'lon': -105}, zoom=10, color='route_color', color_discrete_map='identity', size='size', hover_data='vehicle_label')
    fig.update_layout(mapbox_style='carto-positron')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(showlegend=False)
    fig.update_layout(title={'text': datetime.datetime.fromtimestamp(timestamp).strftime('%B %d %Y, %-I:%M %p'), 'font': {'size': 50}, 'automargin': True, 'yref': 'paper'})
    fig.update_layout(title_x=0.5)
    fig.write_image(f'frames/{timestamp}.png')
    #return fig

In [57]:
start_time = time.time()
prev_time = start_time
for i, timestamp in enumerate(filled_df['vehicle_update_ts_rounded_full'].unique()):    
    print(f'Exporting Frame {timestamp}...')

    export_image_smooth(timestamp)
    
    clear_output(wait=True)

    current_time = time.time()
    print(f'{i+1} / {len(all_timestamps)} done. {current_time - start_time} elapsed, {(current_time - start_time) / (i + 1) * (len(all_timestamps) - i - 1)} remaining')

179 / 3190 done. 311.48196506500244 elapsed, 5239.5094793895105 remaining
Exporting Frame 1718273220...


In [ ]:
os.system("ffmpeg -framerate 24 -pattern_type glob -i 'frames/*.png' -c:v mpeg4 -y -vb 40M movie.mp4")

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

0